In [1]:
import numpy as np
import pandas as pd
from utils.diff import diff

In [2]:
df1 = pd.read_csv('季_资产负债表.csv', usecols=['证券代码', '会计期间', '存货净额', '应收账款净额'], low_memory=False)
# 确保证券代码为整数类型
df1['证券代码'] = pd.to_numeric(df1['证券代码'], errors='coerce').fillna(0).astype(int)

df2 = pd.read_csv('季_利润表.csv', usecols=['证券代码', '会计期间', '营业收入', '利润总额', '管理费用'])
# 确保证券代码为整数类型
df2['证券代码'] = pd.to_numeric(df2['证券代码'], errors='coerce').fillna(0).astype(int)

df3 = pd.merge(df1, df2, on=['证券代码', '会计期间'], how='left')
df3 = diff(df3, col=['营业收入', '利润总额', '管理费用'])

# 77.  sgr
# 季度频率。销售的季度百分比变化。
df3['sgr'] = (df3['营业收入'] - df3.groupby('证券代码').shift(1)['营业收入'])/df3.groupby('证券代码').shift(1)['营业收入'].replace(0, np.nan)

# 54.  pchgm_pchsale
# 季度频率。毛利率变动百分比减去销售变动百分比。
df3['pchgm_pchsale'] = (df3['利润总额'] - df3.groupby('证券代码').shift(1)['利润总额'])/df3.groupby('证券代码').shift(1)['利润总额'].replace(0, np.nan) - df3['sgr']

# 56.  pchsale_pchinvt
# 季度频率。销售的季度变动百分比减去存货的季度变动百分比。
df3['pchsale_pchinvt'] = df3['sgr'] - (df3['存货净额'] - df3.groupby('证券代码').shift(1)['存货净额'])/df3.groupby('证券代码').shift(1)['存货净额'].replace(0, np.nan)

# 57.  pchsale_pchrect
# 季度频率。销售额季度变动百分比减去应收账款季度变动百分比。
df3['pchsale_pchrect'] = df3['sgr'] - (df3['应收账款净额'] - df3.groupby('证券代码').shift(1)['应收账款净额'])/df3.groupby('证券代码').shift(1)['应收账款净额'].replace(0, np.nan)

# 58.  pchsale_pchxsga
# 季度频率。销售的季度变动百分比减去管理费用的季度变动百分比。
df3['pchsale_pchxsga'] = df3['sgr'] - (df3['管理费用'] - df3.groupby('证券代码').shift(1)['管理费用'])/df3.groupby('证券代码').shift(1)['管理费用'].replace(0, np.nan)

df4 = df3[['证券代码', '会计期间', 'pchgm_pchsale', 'pchsale_pchinvt', 'pchsale_pchrect', 'pchsale_pchxsga', 'sgr']]
df4.to_csv('季_54_56_57_58_77.csv', index=False, encoding='utf-8-sig')
df4

,证券代码,会计期间,pchgm_pchsale,pchsale_pchinvt,pchsale_pchrect,pchsale_pchxsga,sgr
0,1,2000-03-31,NaN,NaN,NaN,NaN,NaN
1,1,2000-06-30,NaN,NaN,NaN,NaN,NaN
2,1,2000-09-30,NaN,NaN,NaN,NaN,NaN
3,1,2000-12-31,NaN,NaN,NaN,NaN,NaN
4,1,2001-03-31,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
571126,920118,2023-09-30,NaN,NaN,NaN,NaN,NaN
571127,920118,2023-12-31,NaN,NaN,NaN,NaN,NaN
571128,920118,2024-03-31,NaN,NaN,NaN,NaN,NaN
571129,920118,2024-06-30,NaN,NaN,NaN,NaN,NaN
